In [1]:
import os
import numpy as np
import tensorflow as tf
from model import *
from tensorflow.python import debug as tf_debug

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [31]:
# Misc
LOG_DIR = './log/'

# Training Parameters
learning_rate = 0.001
num_epochs = 1
num_steps = 500
batch_size = 128
display_step = 10

# Network Parameters
num_classes = 10
keep_rate = 0.75

In [32]:
mnist_trn_x = mnist.train.images.reshape([-1, 28, 28, 1])
mnist_trn_y = mnist.train.labels

mnist_val_x = mnist.validation.images.reshape([-1, 28, 28, 1])
mnist_val_y = mnist.validation.labels

mnist_test_x = mnist.test.images.reshape([-1, 28, 28, 1])
mnist_test_y = mnist.test.labels

In [34]:
model_graph = tf.Graph()
with model_graph.as_default():
    
    keep_prob = tf.placeholder(tf.float32)
    
    # Dataset Preparation
    mnist_trn_ds = tf.data.Dataset.from_tensor_slices((mnist_trn_x, mnist_trn_y)).shuffle(500).batch(batch_size)
    mnist_val_ds = tf.data.Dataset.from_tensor_slices((mnist_val_x, mnist_val_y)).shuffle(500).batch(batch_size)
    mnist_test_ds = tf.data.Dataset.from_tensor_slices((mnist_test_x, mnist_test_y)).batch(batch_size)
    
    # String Handle
    handle = tf.placeholder(tf.string, [])
    iterator = tf.data.Iterator.from_string_handle(handle, mnist_trn_ds.output_types, mnist_trn_ds.output_shapes)
    next_element = iterator.get_next()
    
    # Dataset iterators
    training_iterator = mnist_trn_ds.make_initializable_iterator()
    validation_iterator = mnist_val_ds.make_initializable_iterator()
    testing_iterator = mnist_test_ds.make_one_shot_iterator()
    
    # Image Summary
    tf.summary.image('input', next_element[0], 3)
    
    # Logits and Predictions
    logits, embedding_input = build_model(next_element[0], next_element[1], keep_prob) 
    prediction = {'classes': tf.argmax(logits, axis=1), 
                  'prob': tf.nn.softmax(logits, name='prob')}
    
    # Loss and optimizer
    with tf.variable_scope('cross_entropy_loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=next_element[1]))
        tf.summary.scalar('cross_entropy_loss', loss)
    
    with tf.variable_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    
    # Evaluation
    with tf.variable_scope('accuracy'):
        correct_pred = tf.equal(prediction['classes'], tf.argmax(next_element[1], 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
    
    # Global Initializer
    initializer = tf.global_variables_initializer()
    
    # Summary
    summary = tf.summary.merge_all()
    
    # Embedding
    embedding = tf.Variable(tf.zeros([1024, 1024]), name='test_embedding')
    assignment = embedding.assign(embedding_input)
    saver = tf.train.Saver()
    
    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding.name
    embedding_config.metadata_path = 'labels.tsv'

## Use Epoch

In [36]:
with tf.Session(graph=model_graph) as sess:
    
    # Debugger
    #sess = tf_debug.TensorBoardDebugWrapperSession(sess, "MZQ-MBP.local:7000")
    
    # Writer
    writer = tf.summary.FileWriter('./log/model_1', graph=model_graph)
    
    # Embedding projector
    #tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)
    
    # Run Global Initializer
    sess.run(initializer)
    
    # Iterator Handles
    training_handle = sess.run(training_iterator.string_handle())
    validation_handle = sess.run(validation_iterator.string_handle())
    testing_handle = sess.run(testing_iterator.string_handle())
    
    step = 0
    
    for epoch in range(num_epochs):
        
        sess.run(training_iterator.initializer)
        while True:
            # Training
            try:
                sess.run(train_op, feed_dict={keep_prob: keep_rate, handle: training_handle})

                if step % display_step == 0:

                    loss_val, acc, s = sess.run([loss, accuracy, summary],
                                                feed_dict={keep_prob: 1.0, handle: training_handle})

                    writer.add_summary(s, step)

                    print("Epoch " + str(epoch) + ", Step " + str(step) + ", Minibatch Loss= " + \
                          "{:.4f}".format(loss_val) + ", Training Accuracy= " + \
                          "{:.3f}".format(acc))
                    
                step += 1

            except tf.errors.OutOfRangeError:
                break

            # Validation
            if step % (display_step * 10) == 0 and step != 0:

#                 sess.run(assignment, feed_dict={images: mnist.test.images[:1024].reshape([-1, 28, 28, 1]),
#                                                 labels: mnist.test.labels[:1024]})            
#                 saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), step)

                sess.run(validation_iterator.initializer)

                val_acc = 0
                val_loss = 0
                validation_iter = 0

                while True:
                    try:
                        loss_val, acc = sess.run([loss, accuracy], feed_dict={keep_prob: 1.0, handle: validation_handle})
                        val_acc += acc
                        val_loss += loss_val
                        validation_iter += 1
                    except tf.errors.OutOfRangeError:
                        break

                assert validation_iter != 0
                print("Validation" + ", Minibatch Loss= " + \
                      "{:.4f}".format(val_loss/validation_iter) + ", Training Accuracy= " + \
                      "{:.3f}".format(val_acc/validation_iter))


    # Testing
    test_acc = 0
    test_loss = 0
    testing_iter = 0

    while True:        
        try:
            loss_val, acc = sess.run([loss, accuracy], feed_dict={keep_prob: 1.0, handle: testing_handle})
            test_acc += acc
            test_loss += loss_val
            testing_iter += 1
        except tf.errors.OutOfRangeError:
            break

    assert testing_iter != 0
    print("Testing" + ", Minibatch Loss= " + \
          "{:.4f}".format(test_loss/testing_iter) + ", Training Accuracy= " + \
          "{:.3f}".format(test_acc/testing_iter))
            
    writer.close()

Epoch 0, Step 0, Minibatch Loss= 3.1904, Training Accuracy= 0.062
Epoch 0, Step 10, Minibatch Loss= 2.1221, Training Accuracy= 0.375
Epoch 0, Step 20, Minibatch Loss= 1.2844, Training Accuracy= 0.633
Epoch 0, Step 30, Minibatch Loss= 0.7739, Training Accuracy= 0.695
Epoch 0, Step 40, Minibatch Loss= 0.4812, Training Accuracy= 0.844
Epoch 0, Step 50, Minibatch Loss= 0.4937, Training Accuracy= 0.875
Epoch 0, Step 60, Minibatch Loss= 0.4183, Training Accuracy= 0.883
Epoch 0, Step 70, Minibatch Loss= 0.3643, Training Accuracy= 0.898
Epoch 0, Step 80, Minibatch Loss= 0.1484, Training Accuracy= 0.953
Epoch 0, Step 90, Minibatch Loss= 0.0933, Training Accuracy= 0.977
Validation, Minibatch Loss= 0.1877, Training Accuracy= 0.946
Epoch 0, Step 100, Minibatch Loss= 0.2182, Training Accuracy= 0.945
Epoch 0, Step 110, Minibatch Loss= 0.2267, Training Accuracy= 0.922
Epoch 0, Step 120, Minibatch Loss= 0.1527, Training Accuracy= 0.945
Epoch 0, Step 130, Minibatch Loss= 0.1991, Training Accuracy= 0.95

## Use Steps
Don't forget to use:

```
mnist_trn_ds = tf.data.Dataset.from_tensor_slices((mnist_trn_x, mnist_trn_y)).shuffle(500).batch(batch_size)
```

In [12]:
with tf.Session(graph=model_graph) as sess:
    
    # Debugger
    #sess = tf_debug.TensorBoardDebugWrapperSession(sess, "MZQ-MBP.local:7000")
    
    # Writer
    writer = tf.summary.FileWriter('./log/model_1', graph=model_graph)
    
    # Embedding projector
    #tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)
    
    # Run Global Initializer
    sess.run(initializer)
    
    # Iterator Handles
    training_handle = sess.run(training_iterator.string_handle())
    validation_handle = sess.run(validation_iterator.string_handle())
    testing_handle = sess.run(testing_iterator.string_handle())
    
    # Training
    for step in range(num_steps):
        # Training
        try:
            sess.run(train_op, feed_dict={keep_prob: keep_rate, handle: training_handle})

            if step % display_step == 0:

                loss_val, acc, s = sess.run([loss, accuracy, summary],
                                            feed_dict={keep_prob: 1.0, handle: training_handle})

                writer.add_summary(s, step)

                print("Step " + str(step) + ", Minibatch Loss= " + \
                      "{:.4f}".format(loss_val) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc))
                
        except tf.errors.OutOfRangeError:
            break
            
        # Validation
        if step % (display_step * 10) == 0 and step != 0:
            
            sess.run(validation_iterator.initializer)
            
            val_acc = 0
            val_loss = 0
            validation_iter = 0
            
            while True:
                try:
                    loss_val, acc = sess.run([loss, accuracy], feed_dict={keep_prob: 1.0, handle: validation_handle})
                    val_acc += acc
                    val_loss += loss_val
                    validation_iter += 1
                except tf.errors.OutOfRangeError:
                    break
                
            assert validation_iter != 0
            print("Validation" + ", Minibatch Loss= " + \
                  "{:.4f}".format(val_loss/validation_iter) + ", Training Accuracy= " + \
                  "{:.3f}".format(val_acc/validation_iter))
            
#         if step % 100 == 0:
            
#             sess.run(assignment, feed_dict={images: mnist.test.images[:1024].reshape([-1, 28, 28, 1]),
#                                             labels: mnist.test.labels[:1024]})
#             saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), step)


    # Testing
    test_acc = 0
    test_loss = 0
    testing_iter = 0

    while True:        
        try:
            loss_val, acc = sess.run([loss, accuracy], feed_dict={keep_prob: 1.0, handle: testing_handle})
            test_acc += acc
            test_loss += loss_val
            testing_iter += 1
        except tf.errors.OutOfRangeError:
            break

    assert testing_iter != 0
    print("Testing" + ", Minibatch Loss= " + \
          "{:.4f}".format(test_loss/testing_iter) + ", Training Accuracy= " + \
          "{:.3f}".format(test_acc/testing_iter))
            
    writer.close()

Step 0, Minibatch Loss= 2.5452, Training Accuracy= 0.164
Step 10, Minibatch Loss= 1.5652, Training Accuracy= 0.469
Step 20, Minibatch Loss= 0.6957, Training Accuracy= 0.836
Step 30, Minibatch Loss= 0.4986, Training Accuracy= 0.836
Step 40, Minibatch Loss= 0.2813, Training Accuracy= 0.914
Step 50, Minibatch Loss= 0.3131, Training Accuracy= 0.914
Step 60, Minibatch Loss= 0.1990, Training Accuracy= 0.961
Step 70, Minibatch Loss= 0.1807, Training Accuracy= 0.961
Step 80, Minibatch Loss= 0.2785, Training Accuracy= 0.922
Step 90, Minibatch Loss= 0.1874, Training Accuracy= 0.953
Step 100, Minibatch Loss= 0.1178, Training Accuracy= 0.969
Validation, Minibatch Loss= 0.1400, Training Accuracy= 0.962
Step 110, Minibatch Loss= 0.1117, Training Accuracy= 0.977
Step 120, Minibatch Loss= 0.1339, Training Accuracy= 0.961
Step 130, Minibatch Loss= 0.0982, Training Accuracy= 0.969
Step 140, Minibatch Loss= 0.1018, Training Accuracy= 0.961
Step 150, Minibatch Loss= 0.1889, Training Accuracy= 0.938
Step 1

AssertionError: 